In [71]:
using Knet: Knet, dir, accuracy, progress, sgd, load, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout
using Statistics
using Plots
using CUDA
using Random
using LinearAlgebra

LoadError: ArgumentError: Package LinAlg not found in current path:
- Run `import Pkg; Pkg.add("LinAlg")` to install the LinAlg package.


In [110]:
bids=["PA",
  "1C", "1D", "1H", "1S", "1N",
  "2C", "2D", "2H", "2S", "2N",
  "3C", "3D", "3H", "3S", "3N",
  "4C", "4D", "4H", "4S", "4N",
  "5C", "5D", "5H", "5S", "5N",
  "6C", "6D", "6H", "6S", "6N",
  "7C", "7D", "7H", "7S", "7N"];

doubled_score=
[[0   ,  100,  300,  500,  800, 1100, 1400, 1700,
2000, 2300, 2600, 2900, 3200, 3500],
  [0   ,  200,  500,  800, 1100, 1400, 1700, 2000,
    2300, 2600, 2900, 3200, 3500, 3800]];


In [33]:
value(param(2,2))

2×2 KnetArray{Float32,2}:
 -0.615664  -0.890019
  0.68984   -0.164766

Hands are represented with 52-bit vectors and bidding history with 36-bit vectors.

In [106]:
## entries:   (1 2 3 4 5 6 7 8 9 10 11 12 13)--> spades 14 15 16 17 18 19 20 21 22 23 24 25 26
## correspnds (2 3 4 5 6 7 8 9 T  J  Q  K  A).. so on

function random_hand(;remaining=ones(52,1)) #input: remaininn deal,  output: a random hand 
    cards_in_deal=[j for j in 1:52 if remaining[j]==1]
    shuffle!(cards_in_deal)
    hand= [ifelse(j in cards_in_deal[1:13],1,0) for j in 1:52];
    return hand
end

function random_deal(hands=[])
    remaining = ones(52,1);
    all_hands=zeros(52,4)
    for i in 1:length(hands)                #write given pre-dealt hands
            remaining = remaining-hands[i];
            all_hands[:,i]=hands[i]
    end
    generated_hands=4-length(hands)
    for i in 1:generated_hands              #generate remaining hands
        hand=random_hand(remaining=remaining)
        remaining = remaining-hand;
        all_hands[:,i+length(hands)]=hand
    end
    
    return all_hands #returns 52x4 matrix each column is a hand, they are in order as N,S,E,W
end


#features

function hcp_of(hand;suit_seperated=false) #calculates the HCP of a hand both total or suit seperated
    points=[1,2,3,4]
    if suit_seperated
        return [dot(points,hand[10:13]),dot(points,hand[23:26]),dot(points,hand[36:39]),dot(points,hand[49:52])]
    end
    honours=hand[10:13]+hand[23:26]+hand[36:39]+hand[49:52]
    return dot(honours,points)
end

function distribution_of(hand) #returns the card number of each suit in the order S,H,D,C
   return [sum(hand[1:13]),sum(hand[14:26]),sum(hand[27:39]),sum(hand[40:52])] 
end

#tools

function human_readable(hand)
    to_cards=["2","3","4","5","6","7","8","9","T","J","Q","K","A"]
    suits=[]
    for i in 1:4
        suit=hand[13*(i-1)+1:13*i]
        cards= [to_cards[j] for j in 13:-1:1 if suit[j]==1]
        push!(suits,join(cards))
    end
    return suits
end


human_readable (generic function with 1 method)

In [108]:
a=random_hand()
hcp_of(a,suit_seperated= false)
human_readable(a)

4-element Array{Any,1}:
 "QJT2"
 "5"
 "AKQ4"
 "AQT7"

In [42]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = loss(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

struct Dense
    w;
    b;
    f;
end

Dense(i::Int,o::Int,f=relu) = Dense(param(o,i),param0(o),f)
(l::Dense)(x) = l.f.(l.w * x .+ l.b)

dnn_init=Chain(Dense(52,128),Dense(128,128),Dense(128,128),Dense(128,36,identity));
dnn=Chain(Dense(52+36,128),Dense(128,128),Dense(128,128),Dense(128,36,identity));
println.(summary.((l.w,l.b)) for l in dnn_init.layers);


("128×52 Param{KnetArray{Float32,2}}", "128-element Param{KnetArray{Float32,1}}")
("128×128 Param{KnetArray{Float32,2}}", "128-element Param{KnetArray{Float32,1}}")
("128×128 Param{KnetArray{Float32,2}}", "128-element Param{KnetArray{Float32,1}}")
("36×128 Param{KnetArray{Float32,2}}", "36-element Param{KnetArray{Float32,1}}")
